# Upload synthetics to dandi

author: laquitainesteeve@gmail.com

Execution time: 10 min

Special hardware: on CPU, does not require GPU.

# Setup 

Activate demo virtual environment (envs/demo.yml)

```bash
python -m ipykernel install --user --name demo --display-name "demo"
```

In [17]:
%%time 

# import python packages
import os
import yaml
import numpy as np
from time import time
from dandi.dandiapi import DandiAPIClient
import spikeinterface.extractors as se
import spikeinterface.sorters as ss
import spikeinterface
import spikeinterface as si
from pynwb.file import NWBFile, Subject
from pynwb import NWBHDF5IO
import uuid
from datetime import datetime
from dateutil.tz import tzlocal
from neuroconv.tools.spikeinterface import add_recording_to_nwbfile, add_sorting_to_nwbfile
print("spikeinterface", spikeinterface.__version__)

# setup project
proj_path = "/home/steeve/steeve/epfl/code/spikebias/"
os.chdir(proj_path)

from src.nodes.dandi import write_nwb

# setup pipeline parameters
DANDISET_ID = '001250'

# setup nwb files save paths
FILE_PATH = os.path.join(proj_path, "temp/pros_of_details/")

spikeinterface 0.101.2
CPU times: user 328 μs, sys: 0 ns, total: 328 μs
Wall time: 394 μs


## Custom functions

In [6]:
def upload_recording_and_ground_truth_nwb_to_dandi(dandiset_id: str, subject_path: str, file_path: str):
    """Upload the recording and ground truth nwb datasets 
    to dandi archive
    """

    # download a minimal local dandiset with only the .yaml config
    os.system(f"dandi download --download dandiset.yaml --output-dir {file_path} DANDI:{dandiset_id}")

    # go to path of the dandiset
    os.chdir(file_path)

    # upload
    os.system(
        f"""
        export DANDI_API_KEY='210e68743286d64e84743bd8980d5771ef82bf4d';
        cd {os.path.join(file_path, dandiset_id)};
        dandi organize {subject_path} -f dry;
        dandi organize {subject_path};    
        dandi upload
        """
    )

    # delete local dandiset
    os.system(f"rm -rf {os.path.join(file_path, dandiset_id)}")

## Upload buccino replicate

- This replicates Buccino et al., simulation with MEAREC as described in the Spikeinterface paper
- Storage: 2.8 GB on dandi
- Execution time: 16 min

In [18]:
%%time 

# setup parameters
SUBJECT_ID = "mearec-synthetics-buccino-rep"
SESSION_ID = 'buccino_rep'
SESSION_DESCRIPTION = 'Replication of Buccino synthetic model'        
RECORDING_PATH = 'dataset/00_raw/recording_buccino_rep/'
GROUND_TRUTH_PATH = 'dataset/00_raw/ground_truth_buccino_rep/'
SUBJECT_PATH = os.path.join(FILE_PATH, f"{DANDISET_ID}/sub-{SUBJECT_ID}/")
NWB_FILE_PATH = os.path.join(SUBJECT_PATH, f"sub-{SUBJECT_ID}_ses-{SESSION_ID}.nwb")
PARAMS = {
    "subject_id": SUBJECT_ID,
    "session_description": SESSION_DESCRIPTION,
    "identifier": str(uuid.uuid4()),
    "session_start_time": datetime.now(tzlocal()),
    "experimenter": "Laquitaine Steeve",
    "lab": "Blue Brain Project",
    "institution": "EPFL",
    "experiment_description": SESSION_DESCRIPTION,
    "session_id": SESSION_ID,
    "related_publications": "doi:"
    }

# load extractors
RecordingS1 = si.load_extractor(RECORDING_PATH)
SortingGtS1 = si.load_extractor(GROUND_TRUTH_PATH)
    
# write as nwb
write_nwb.run(RECORDING_PATH, GROUND_TRUTH_PATH, NWB_FILE_PATH, PARAMS)

# upload
upload_recording_and_ground_truth_nwb_to_dandi(DANDISET_ID, SUBJECT_PATH, FILE_PATH)

2025-08-13 16:21:17,386 - root - write_nwb.py - run - INFO - Ground truth metadata:
2025-08-13 16:21:17,387 - root - write_nwb.py - run - INFO - ['z', 'y', 'bursting', 'mtype', 'exp_decay', 'x', 'max_burst_duration', 'max_spikes_per_burst', 'cell_type', 'snr']


/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.5 is recommended
  warnings.warn(
/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/neuroconv/tools/spikeinterface/spikeinterface.py:1054: UserWarning: The 'add_recording' function is deprecated and will be removed after February 2025. 
Please use 'add_recording_to_nwbfile' instead.
  warnings.warn(
/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/neuroconv/tools/spikeinterface/spikeinterface.py:1676: UserWarning: The 'add_sorting' function is deprecated and will be removed after February 2025. 
Please use 'add_sorting_to_nwbfile' instead.
  warnings.warn(


2025-08-13 16:21:18,047 - root - utils.py - create_if_not_exists - INFO - The following path has been created /home/steeve/steeve/epfl/code/spikebias/temp/pros_of_details/001250/sub-mearec-synthetics-buccino-rep
PATH                 SIZE DONE    DONE% CHECKSUM STATUS MESSAGE   
001250/dandiset.yaml                             done   updated   
Summary:                  0 Bytes                1 done 1 updated 
                          <0.00%                                  


2025-08-13 16:21:35,234 [    INFO] Logs saved in /home/steeve/.local/state/dandi-cli/log/2025.08.13-14.21.33Z-3459053.log
2025-08-13 16:21:36,071 [    INFO] Loading metadata from 1 files
2025-08-13 16:21:36,189 [    INFO] Organized 0 out of 1 paths. Visit /home/steeve/steeve/epfl/code/spikebias/temp/pros_of_details/001250/
2025-08-13 16:21:36,189 [    INFO] Logs saved in /home/steeve/.local/state/dandi-cli/log/2025.08.13-14.21.35Z-3459077.log


DRY: /home/steeve/steeve/epfl/code/spikebias/temp/pros_of_details/001250/sub-mearec-synthetics-buccino-rep/sub-mearec-synthetics-buccino-rep_ses-buccino_rep.nwb -> sub-mearec-synthetics-buccino-rep/sub-mearec-synthetics-buccino-rep_ecephys.nwb


2025-08-13 16:21:37,068 [    INFO] Loading metadata from 1 files
2025-08-13 16:21:37,069 [    INFO] Symlink support autodetected; setting files_mode='symlink'
2025-08-13 16:21:37,072 [    INFO] Organized 1 paths. Visit /home/steeve/steeve/epfl/code/spikebias/temp/pros_of_details/001250/
2025-08-13 16:21:37,072 [    INFO] Logs saved in /home/steeve/.local/state/dandi-cli/log/2025.08.13-14.21.36Z-3459156.log
2025-08-13 16:21:38,629 [    INFO] Found 3 files to consider


PATH                                                                                  SIZE    ERRORS  PROGRESS STATUS                MESSAGE                                                            
dandiset.yaml                                                                         5.6 kB                   skipped               should be edited online                                            
sub-mearec-synthetics-buccino-rep/sub-mearec-synthetics-buccino-rep_ecephys.nwb       29.5 GB   0         100% done                  exists (older) - reuploading                                       
...earec-synthetics-buccino-rep/sub-mearec-synthetics-buccino-rep_ses-buccino_rep.nwb 29.5 GB   0         100% ERROR                 Error 409 while sending POST request to https://api.dandiarchive...
Summary:                                                                              59.1 GB        60.3 MB/s 1 skipped             1 should be edited online                                      

2025-08-13 16:37:58,961 [    INFO] Logs saved in /home/steeve/.local/state/dandi-cli/log/2025.08.13-14.21.37Z-3459237.log
Error: Error 409 while sending POST request to https://api.dandiarchive.org/api/uploads/26d2f3d1-baf4-4dd7-9322-0625e727c9bb/validate/: "An identical blob has already been uploaded."


CPU times: user 7.28 s, sys: 8.63 s, total: 15.9 s
Wall time: 16min 43s


### Upload S2

- Cell density and count matched to biophy-model (validated on in-vivo)
- storage: 25 GB on dandi
- Execution time: 9m

In [16]:
%%time 

# setup parameters
SUBJECT_ID = "mearec-synthetics-s2"
SESSION_ID = 's2_cell_density_and_count'
SESSION_DESCRIPTION = 'Simulation of buccino with in-vivo cell density and count'        
RECORDING_PATH = 'dataset/00_raw/recording_synth2/'
GROUND_TRUTH_PATH = 'dataset/00_raw/ground_truth_synth2/'
SUBJECT_PATH = os.path.join(FILE_PATH, f"{DANDISET_ID}/sub-{SUBJECT_ID}/")
NWB_FILE_PATH = os.path.join(SUBJECT_PATH, f"sub-{SUBJECT_ID}_ses-{SESSION_ID}.nwb")
PARAMS = {
    "subject_id": SUBJECT_ID,
    "session_description": SESSION_DESCRIPTION,
    "identifier": str(uuid.uuid4()),
    "session_start_time": datetime.now(tzlocal()),
    "experimenter": "Laquitaine Steeve",
    "lab": "Blue Brain Project",
    "institution": "EPFL",
    "experiment_description": SESSION_DESCRIPTION,
    "session_id": SESSION_ID,
    "related_publications": "doi:"
    }

# load extractors
RecordingS1 = si.load_extractor(RECORDING_PATH)
SortingGtS1 = si.load_extractor(GROUND_TRUTH_PATH)
    
# write as nwb
write_nwb.run(RECORDING_PATH, GROUND_TRUTH_PATH, NWB_FILE_PATH, PARAMS)

# upload
upload_recording_and_ground_truth_nwb_to_dandi(DANDISET_ID, SUBJECT_PATH, FILE_PATH)

2025-08-13 15:40:28,633 - root - write_nwb.py - run - INFO - Ground truth metadata:
2025-08-13 15:40:28,633 - root - write_nwb.py - run - INFO - ['z', 'y', 'bursting', 'mtype', 'exp_decay', 'x', 'max_burst_duration', 'max_spikes_per_burst', 'cell_type', 'snr']


/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.5 is recommended
  warnings.warn(
/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/neuroconv/tools/spikeinterface/spikeinterface.py:1054: UserWarning: The 'add_recording' function is deprecated and will be removed after February 2025. 
Please use 'add_recording_to_nwbfile' instead.
  warnings.warn(
/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/neuroconv/tools/spikeinterface/spikeinterface.py:1676: UserWarning: The 'add_sorting' function is deprecated and will be removed after February 2025. 
Please use 'add_sorting_to_nwbfile' instead.
  warnings.warn(


2025-08-13 15:40:40,339 - root - utils.py - create_if_not_exists - INFO - The following path has been created /home/steeve/steeve/epfl/code/spikebias/temp/pros_of_details/001250/sub-mearec-synthetics-s2


2025-08-13 15:41:00,648 [    INFO] Logs saved in /home/steeve/.local/state/dandi-cli/log/2025.08.13-13.40.58Z-3429200.log


PATH                 SIZE DONE    DONE% CHECKSUM STATUS MESSAGE   
001250/dandiset.yaml                             done   updated   
Summary:                  0 Bytes                1 done 1 updated 
                          <0.00%                                  


...

DRY: /home/steeve/steeve/epfl/code/spikebias/temp/pros_of_details/001250/sub-mearec-synthetics-s2/sub-mearec-synthetics-s2_ses-s2_cell_density_and_count.nwb -> sub-mearec-synthetics-s2/sub-mearec-synthetics-s2_ecephys.nwb


2025-08-13 15:41:02,433 [    INFO] Loading metadata from 1 files
2025-08-13 15:41:02,434 [    INFO] Symlink support autodetected; setting files_mode='symlink'
2025-08-13 15:41:02,437 [    INFO] Organized 1 paths. Visit /home/steeve/steeve/epfl/code/spikebias/temp/pros_of_details/001250/
2025-08-13 15:41:02,437 [    INFO] Logs saved in /home/steeve/.local/state/dandi-cli/log/2025.08.13-13.41.01Z-3429293.log
2025-08-13 15:41:04,026 [    INFO] Found 3 files to consider


PATH                                                                                SIZE       ERRORS      PROGRESS STATUS                MESSAGE                  
dandiset.yaml                                                                       5.6 kB                          skipped               should be edited online  
sub-mearec-synthetics-s2/sub-mearec-synthetics-s2_ecephys.nwb                       29.6 GB       0            100% done                                           
sub-mearec-synthetics-s2/sub-mearec-synthetics-s2_ses-s2_cell_density_and_count.nwb 29.6 GB       1                 ERROR                 failed validation        
Summary:                                                                            59.1 GB 1 with errors 56.8 MB/s 1 skipped             1 should be edited online
                                                                                                                    1 done                1 failed validation      
                

2025-08-13 15:49:44,165 [ WARNING] One or more assets failed validation.  Consult the logfile for details.
2025-08-13 15:49:44,365 [    INFO] Logs saved in /home/steeve/.local/state/dandi-cli/log/2025.08.13-13.41.02Z-3429372.log
Error: failed validation


CPU times: user 21.9 s, sys: 8.17 s, total: 30.1 s
Wall time: 9min 17s


In [ ]:
# def standardize_metadata(GroundTruth):
#     """standardize ground truth extractor metadata
#     from mearec simulatio

#     Args:
#         GroundTruth (_type_): _description_

#     Returns:
#         _type_: _description_
#     """
#     GroundTruth.set_property('x', GroundTruth.get_property('soma_position')[:,0])
#     GroundTruth.set_property('y', GroundTruth.get_property('soma_position')[:,1])
#     GroundTruth.set_property('z', GroundTruth.get_property('soma_position')[:,2])
#     GroundTruth.delete_property('soma_position')

#     GroundTruth.set_property('bursting', GroundTruth.get_property('bursting').astype(int))
#     GroundTruth.set_property('exp_decay', GroundTruth.get_property('exp_decay').astype(str))
#     GroundTruth.set_property('max_burst_duration', GroundTruth.get_property('max_burst_duration').astype(str))
#     GroundTruth.set_property('max_spikes_per_burst', GroundTruth.get_property('max_spikes_per_burst').astype(str))
#     return GroundTruth

# GROUND_TRUTH_PATH = 'dataset/00_raw/ground_truth_synth2/'

# SortingS2 = si.load_extractor(GROUND_TRUTH_PATH)

# SortingS2 = standardize_metadata(SortingS2)

# SortingS2.save(folder=GROUND_TRUTH_PATH, overwrite=True)

/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.5 is recommended
  warnings.warn(
